# 13장 빅데이터

# 13.4 파이스파크

## `myfile.txt`가 주어지지 않아서 실행은 안됩니다.

In [ ]:
# SparkContext 객체 생성
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext(conf=conf)

# 파일의 각 행을 읽고 클러스터로 분산시켜서
# Spark RDD를 만듦
lines = open("myfile.txt")
lines_rdd = sc.parallelize(lines)

# 문장 부호를 없애고 모든 행을 소문자로 변환
def clean_line(s):
    s2 = s.strip().lower()
    s3 = s2.replace(".","").replace(",","")
    return s3

lines_clean = lines_rdd.map(clean_line)

# 각 행을 단어 단위로 쪼갬
words_rdd = lines_clean.flatmap(lambda l: l.split())

# 각 단어의 등장 횟수를 셈
def merge_counts(count1, count2):
    return count1 + count2

words_w_1 = words_rdd.map(lambda w: (w, 1))
counts = words_w_1.reduceByKey(merge_counts)

# 횟수를 취합하여 출력
for word, count in counts.collect():
    print("%s: %i " % (word, count))

# 13.11

In [ ]:
def mapper(line):
    l2 = l.strip().lower()
    l3  = l2.replace(".","").replace(",","")
    words = l3.split()
    return [(w, 1) for w in words]

def reducer_func(count1, count2):
    return count1 + count2

lines = open("myfile.txt")
lines_rdd = sc.parallelize(lines)

map_stage_out = lines_rdd.flatMap(mapper)
reduce_stage_out = map_stage_out.reduceByKey(reducer_func)

# 13.12

In [ ]:
def mapper(line):
    return [(w, [w]) for w in line.split()]

def red_func(lst1, lst2):
    return lst1 + lst2

result = lines.flatMap(mapper).reduceByKey(red_func)


In [ ]:
def update_agg(agg_list, new_word):
    agg_list.append(new_word)
    return agg_list  # 동일한 리스트!

def merge_agg_lists(agg_list1, agg_list2):
    return agg_list1 + agg_list2

result = lines.flatMap(mapper).aggregateByKey(
    [], update_agg, merge_agg_lists)
